# Embeddings und Vektordatenbanken

Embeddings sind zentral in Anwendungen von großen Sprachmodellen (large language models LLMs). Im Wesentlichen handelt es sich dabei um dichte Vektorrepräsentationen von Wörtern, die semantische Beziehungen erfassen.

#### Training und Funktion

Embeddings werden durch unüberwachtes Lernen an großen Textkorpora trainiert, wobei Wörter mit ähnlichen Bedeutungen im Vektorraum nah beieinander positioniert werden.

#### Nutzen

Wort-Embeddings verbessern das Sprachverständnis des Modells, indem sie Wissen über Beziehungen zwischen Wörtern liefern, was genauere Vorhersagen und kontextbewusstere Antworten ermöglicht. Diese Embeddings ermöglichen ein gewisses Maß an semantischem Verständnis.

In [ ]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv  # zum laden von environment variablen
from langchain_openai import AzureOpenAIEmbeddings
from sklearn.metrics.pairwise import (
    cosine_similarity,  # zum vergleichen zweier Vektoren
)
load_dotenv()

Wir verwenden hier ein Embedding-Modell von OpenAI:

In [17]:
def get_embedder() -> AzureOpenAIEmbeddings:
    """Obtain an instance of the OpenAIEmbeddings class - a wrapper around the OpenAI API embedding functionality.

    Returns:
        OpenAIEmbeddings: an instance of the OpenAIEmbeddings class
    """
    return AzureOpenAIEmbeddings(model="text-embedding-3-small")


Ein häufig verwendetes Modell ist "text-embedding-ada-002", wir verwenden den Nachfolger "text-embedding-3":

In [18]:
embedder = get_embedder()

In [ ]:
embeddings = embedder.embed_query("This is a test query")

sns.lineplot(x=range(len(embeddings)), y=embeddings)
plt.ylabel('Embedding Value')
plt.xlabel('Dimension')
plt.title('Embeddings')
plt.show()

Nun können wir beliebige Texte "embedden", und die Ähnlichkeit von zwei Texter via ihre embeddings vergleichen.

In [ ]:
embedding_1 = embedder.embed_query("This is a test query")
embedding_2 = embedder.embed_query("This is another test query")

similarity_score = cosine_similarity([embedding_1], [embedding_2])[0, 0]

print(f"The similarity score is {similarity_score}")

## Aufgaben

1.1. Sehen Sie sich Embeddings anderer Eingaben an. Wie sehen diese aus? Welche Dinge sind tendenziell ähnlich/unähnlich? Können Sie längere Texte embedden, und macht das einen Unterschied? Können Sie andere Maße finden, die die Ähnlichkeit zwischen Embeddings bewerten?

1.2. Können Sie den obigen Code erweitern, um ein Embedding für verschiedene Textsorten zu erstellen und einen "maximal ähnlichen" Text für eine spezifische Texteingabe zu finden?

# Vector stores und Similarity search

Ein Vectorstore (oder Vektordatenbank) ist eine spezialisierte Datenbank, die darauf ausgelegt ist, semantische Suche von Texten mithilfe von Vektor-Embeddings zu ermöglichen.


In [ ]:
from uuid import uuid4

from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore

document_1 = Document(page_content="Alice likes pizza.", metadata={"page": 1})
document_2 = Document(page_content="Bob loves pasta.", metadata={"page": 2})
document_3 = Document(page_content="This is a test document.", metadata={"page": 1})
document_4 = Document(page_content="This is another test document.", metadata={"page": 1})

vectorstore = InMemoryVectorStore(embedding=get_embedder())

documents = [document_1, document_2, document_3, document_4]
ids = [str(uuid4()) for _ in range(len(documents))]
vectorstore.add_documents(documents=documents, ids=ids)

Wir können die Vektordatenbank nun in einen "retriever" (quasi eine Suchmaschine) umwandeln.

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2, "fetch_k": 2, "lambda_mult": 0.5},
)
retriever.invoke("bob")

# Aufgaben

1.3. Welche Such-Parameter gibt es und was tun diese? Wie bekommen wir möglichst verschiedene Dokumente (in dem Fall, dass wir viele ähnliche Dokumente in der Datenbank haben)?

1.4. Wie können wir nur die relevantesten Dokumente erhalten?


# Ingest

Der "Ingest" Schritt bezeichnet das Laden und vorbereiten der "knowledge base" des RAG systems. Hierbei werden wir langchain Funktionen verwenden um ein oder meherere PDFs einzulesen, aufzuspalten und zusammein mit ihren embeddings in eine Vektordatenbank zu speichen.

## Aufgabe

1.5. Laden das angegebene PDF in eine Vektordatenbank.
1.6. Teste die Suche mit der Vektordatenbank. Schaue dafür in das PDF und gib einen Suchbegriff ein.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("../data/Broschure-Sortiment-MicroFluidics.pdf")
# TODO: Lade das PDF-Dokument und weise es der Variable pdf_pages zu.
pdf_pages = None

In [ ]:
print(f'Content: \n {pdf_pages[0].page_content}')
print('##########')
print(f'Source: \n {pdf_pages[0].metadata["source"]}')
print('##########')
print(f'Page: \n {pdf_pages[0].metadata["page"]}')